In [1]:
import os 
import hashlib
import pandas as pd
import torch
from torch import cat

hashfn = lambda s : hashlib.md5(s.encode()).hexdigest()
pad = lambda x : cat([x, zeros(513-x.shape[0],x.shape[1]).to(x.device)], 
                     dim=0)

emb_dir = '../seq-smiles-embeds/'
emb_paths = [os.path.join(emb_dir, i) for i in os.listdir(emb_dir)]
df = pd.read_csv('../data/seq-smiles-hash.csv')
df

,seq,smiles,hit,hash
0,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,True,c745b067d9f32b2f48d85561cac66cbb
1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,True,c745b067d9f32b2f48d85561cac66cbb
2,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,CC(C)=CCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@...,True,c745b067d9f32b2f48d85561cac66cbb
3,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,True,c745b067d9f32b2f48d85561cac66cbb
4,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,True,c745b067d9f32b2f48d85561cac66cbb
...,...,...,...,...
1587668,MAINAQEISALIKQQIENFKPNFDVTETGVVTYIGDGIARAHGLEN...,CCOC1=C2C3=C(CC4C5C3(CCN4C)C(O2)C(C=C5)O)C=C1,True,62c88cf9cbc4f26c69679044f42ae957
1587669,MAINAQEISALIKKQIENFQPNFDVTETGIVTYIGDGIARARGLDN...,CC(=O)OC1CCC2C3CCC4(C(C3CCC2=C1)CCC4(C#C)OC(=O...,True,ffdc52fa4f5fcc14981939cb3a568212
1587670,MAINAQEISALIKKQIENFQPNFDVTETGIVTYIGDGIARARGLDN...,CCOC1=C2C3=C(CC4C5C3(CCN4C)C(O2)C(C=C5)O)C=C1,True,ffdc52fa4f5fcc14981939cb3a568212
1587671,MAINAQEISALIKKQIENFQPNFDVTETGIVTYIGDGIARARGLDN...,CC(=O)OC1CCC2C3CCC4(C(C3CCC2=C1)CCC4(C#C)OC(=O...,True,493fba3415903942d4f848cab4777520


In [2]:
print(f'n data points: {len(df)}')
print(f'n unique data points: {len(df.drop_duplicates())}')
print(f'frac unique: {len(df.drop_duplicates())/len(df)}')

print(f'n unique sequence: {len(df["seq"].unique())}')
print(f'frac unique sequence: {len(df["seq"].unique()) / len(df)}')

print(f'n unique smiles: {len(df["smiles"].unique())}')
print(f'frac unique smiles: {len(df["smiles"].unique())/len(df)}')

n data points: 1587673
n unique data points: 707285
frac unique: 0.44548531089210436
n unique sequence: 17126
frac unique sequence: 0.010786855983568405
n unique smiles: 425672
frac unique smiles: 0.2681106247949042


In [5]:
%%time
fps = pd.read_csv('../data/seq-smiles.fp.csv', index_col=0, header=None)
fps

CPU times: user 2min 12s, sys: 13.7 s, total: 2min 25s
Wall time: 2min 25s


,1,2,3,4,5,6,7,8,9,10,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,,,,,,,,,,,,,,,,,,,,,
OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCC)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CC(C)=CCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCO)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC1=C(C2=CC3=NC(=CC4=C(C(=CO)C(=N4)C=C5C(=C(C(=N5)C=C1N2)C(CCC=C(C)CCC=C(C)CCC=C(C)C)O)C)CCC(=O)O)C(=C3C)CCC(=O)O)C=C,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
C[N+]1=CC=CC=C1C(=O)O,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
CCCCCCCCC=CCCCCCCCC#N,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
%%time
fts = pd.read_csv('../data/seq-smiles-ft.csv', index_col=0)
fts

CPU times: user 1.06 s, sys: 28.3 ms, total: 1.09 s
Wall time: 1.09 s


,CalcExactMolWt,CalcNumLipinskiHBD,CalcNumLipinskiHBA,CalcNumHBD,CalcNumHBA,CalcNumRotatableBonds,CalcNumRings,CalcNumAromaticRings,CalcNumSaturatedRings,CalcNumHeterocycles,CalcNumAromaticHeterocycles,CalcNumAromaticCarbocycles,CalcNumSaturatedCarbocycles,CalcNumAliphaticRings,CalcNumAliphaticHeterocycles,CalcNumHeteroatoms,CalcNumAmideBonds,CalcFractionCSP3
0,,,,,,,,,,,,,,,,,,
OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C1=O,480.298808,3.0,6.0,3.0,4.0,15.0,4.0,2.0,2.0,1.0,0.0,2.0,1.0,2.0,1.0,6.0,2.0,0.551724
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCC)C1=O,438.288243,2.0,5.0,2.0,3.0,14.0,3.0,2.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,5.0,2.0,0.518519
CC(C)=CCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,462.288243,2.0,5.0,2.0,3.0,14.0,3.0,2.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,5.0,2.0,0.413793
OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCO)C1=O,498.309372,4.0,7.0,4.0,5.0,18.0,3.0,2.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,7.0,2.0,0.551724
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,450.288243,2.0,5.0,2.0,3.0,14.0,3.0,2.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,5.0,2.0,0.464286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC1=C(C2=CC3=NC(=CC4=C(C(=CO)C(=N4)C=C5C(=C(C(=N5)C=C1N2)C(CCC=C(C)CCC=C(C)CCC=C(C)C)O)C)CCC(=O)O)C(=C3C)CCC(=O)O)C=C,798.435636,5.0,10.0,5.0,9.0,26.0,5.0,1.0,0.0,5.0,1.0,0.0,0.0,4.0,4.0,10.0,0.0,0.367347
C[N+]1=CC=CC=C1C(=O)O,138.054955,1.0,3.0,1.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.142857
CCCCCCCCC=CCCCCCCCC#N,263.261300,0.0,1.0,0.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.833333


In [7]:
%%time
import umap

embedding = umap.UMAP().fit_transform(fps.values)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [9]:
fps.dropna()

,1,2,3,4,5,6,7,8,9,10,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,,,,,,,,,,,,,,,,,,,,,
OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCC)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CC(C)=CCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCO)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C(C)C)C1=O,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC1=C(C2=CC3=NC(=CC4=C(C(=CO)C(=N4)C=C5C(=C(C(=N5)C=C1N2)C(CCC=C(C)CCC=C(C)CCC=C(C)C)O)C)CCC(=O)O)C(=C3C)CCC(=O)O)C=C,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
C[N+]1=CC=CC=C1C(=O)O,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
CCCCCCCCC=CCCCCCCCC#N,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
